In [104]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm


data_folder = os.path.join(os.getcwd(), '../data_original')
df = pd.read_csv(os.path.join(data_folder, 'correct_data.csv'), sep=',')

In [105]:
df.head()

,Date/heure transaction,ID acheteur,Article,Famille d'article,Prix unitaire TTC,Quantité,Total TTC,Periode,Jour semaine,Semestre,Automne/Printemps
0,2014-05-02 18:43:56,9447,oasis tropical,Softs,0.7,1.0,0.7,apresmidi,vendredi,P14,P
1,2014-05-02 18:53:05,9453,coca,Softs,0.7,4.0,2.8,apresmidi,vendredi,P14,P
2,2014-05-02 18:53:05,9453,ice tea peche,Softs,0.7,1.0,0.7,apresmidi,vendredi,P14,P
3,2014-05-02 18:53:44,9701,ice tea peche,Softs,0.7,1.0,0.7,apresmidi,vendredi,P14,P
4,2014-05-02 18:56:57,9603,coca,Softs,0.7,1.0,0.7,apresmidi,vendredi,P14,P


In [106]:
df.drop(columns= ['Date/heure transaction', 'Article', 'Prix unitaire TTC', 'Jour semaine', 'Semestre', 'Automne/Printemps'], axis=1, inplace=True)
df.columns

Index(['ID acheteur', 'Famille d'article', 'Quantité', 'Total TTC', 'Periode'], dtype='object')

In [107]:
df['ID acheteur'].nunique()

10692

In [108]:
# Initialisation du nouveau dataframe
df_resultat = pd.DataFrame({'ID acheteur': df['ID acheteur'].unique()})

# Création des combinaisons de famille et période
combinations = pd.MultiIndex.from_product([df['Famille d\'article'].unique(), df['Periode'].unique()], names=['Famille d\'article', 'Periode'])
combinations_names = {(famille, periode) : f"{famille}_{periode}" for famille, periode in combinations}

# Ajout des colonnes de famille_période au dataframe résultat
df_resultat = df_resultat.reindex(columns=[*df_resultat.columns.to_list(), *combinations_names.values()])

# Groupement et agrégation des données
grouped = df.groupby(['ID acheteur', 'Famille d\'article', 'Periode'])['Quantité'].sum().reset_index()

# to csv

# Fusion des données agrégées dans le dataframe résultat
for row, i in zip(grouped.itertuples(), tqdm(range(len(grouped)))):
    df_resultat.loc[df_resultat['ID acheteur'] == row[1], combinations_names[(row[2], row[3])]] = row[4]

# Remplacement des valeurs manquantes par 0
df_resultat = df_resultat.fillna(0)


100%|█████████▉| 106814/106815 [00:20<00:00, 5117.27it/s]


In [109]:
# Création des combinaisons de famille et période
prices_names = [f"prix_{periode}" for periode in df['Periode'].unique()]

# Ajout des colonnes de prices_names au dataframe résultat
df_resultat = df_resultat.reindex(columns=[*df_resultat.columns.tolist(), *prices_names])

# Groupement et agrégation des données
grouped = df.groupby(['ID acheteur', 'Periode'])['Total TTC'].mean().reset_index()

# Fusion des données agrégées dans le dataframe résultat
for row, i in zip(grouped.itertuples(), tqdm(range(len(grouped)))):
    df_resultat.loc[df_resultat['ID acheteur'] == row[1], f'prix_{row[2]}'] = row[3]

# Remplacement des valeurs manquantes par 0
df_resultat = df_resultat.fillna(0)


100%|█████████▉| 31041/31042 [00:06<00:00, 5049.75it/s]


In [110]:
df_resultat.head()

,ID acheteur,Softs_apresmidi,Softs_soir,Softs_matin,Softs_midi,Snacks Sucrés_apresmidi,Snacks Sucrés_soir,Snacks Sucrés_matin,Snacks Sucrés_midi,Pampryls_apresmidi,...,Bieres_matin,Bieres_midi,Fruits & Légumes_apresmidi,Fruits & Légumes_soir,Fruits & Légumes_matin,Fruits & Légumes_midi,prix_apresmidi,prix_soir,prix_matin,prix_midi
0,9447,43.0,4.0,11.0,40.0,20.0,3.0,8.0,13.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.939688,1.937261,0.745349,0.706111
1,9453,124.0,21.0,61.0,127.0,41.0,5.0,11.0,33.0,15.0,...,0.0,5.0,0.0,0.0,0.0,0.0,1.149896,1.792669,0.686705,0.698964
2,9701,53.0,21.0,4.0,46.0,36.0,0.0,3.0,12.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.811579,1.657937,0.724194,0.727895
3,9603,21.0,4.0,17.0,21.0,21.0,0.0,2.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.163214,1.781368,0.662500,0.607143
4,12024,4.0,4.0,0.0,9.0,9.0,6.0,5.0,19.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.731818,1.024500,0.813043,0.558667


In [113]:
df_resultat.to_csv(os.path.join(data_folder, 'users.csv'), index=False)